<a href="https://colab.research.google.com/github/CharithaGongati/programming-quiz/blob/main/quizprogram2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import Adam, SGD
from keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

# Step 1: Load the MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32') / 255
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype('float32') / 255
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

# Step 2: Define the model creation function for GridSearch
def create_model(optimizer=Adam(), learn_rate=0.01):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(10, activation='softmax'))

    if optimizer == 'Adam':
        optimizer = Adam(learning_rate=learn_rate)
    elif optimizer == 'SGD':
        optimizer = SGD(learning_rate=learn_rate)

    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Step 3 and 5: Grid search for the best hyperparameters
model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=32, verbose=0)
optimizer = ['SGD', 'Adam']
learn_rate = [0.001, 0.01]
param_grid = dict(optimizer=optimizer, learn_rate=learn_rate, batch_size=[32, 64])
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

# Summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# Step 6: Use callbacks and retrain the best model
best_optimizer = grid_result.best_params_['optimizer']
best_learn_rate = grid_result.best_params_['learn_rate']
best_batch_size = grid_result.best_params_['batch_size']

model = create_model(optimizer=best_optimizer, learn_rate=best_learn_rate)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=best_batch_size, callbacks=[reduce_lr], verbose=2)

# Evaluate the model
scores = model.evaluate(X_test, y_test, verbose=0)
print('Final CNN accuracy:', scores[1])

# Plot training & validation accuracy values
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

ImportError: cannot import name 'np_utils' from 'keras.utils' (/usr/local/lib/python3.10/dist-packages/keras/utils/__init__.py)